In [17]:
#Imports of all necessary Python Packages
import sklearn as skl
import numpy as np
import pandas as pd
from time import time
import csv

#Classifiers
from sklearn.svm import SVC

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis 
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from sklearn.decomposition import PCA

from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import VotingClassifier

from sklearn.linear_model import LogisticRegressionCV

from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV

#Evaluation
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn import cross_validation
from sklearn.cross_validation import train_test_split

In [6]:
man_feats = pd.read_csv('regras.csv', index_col=0)


In [7]:
pos_tags = pd.read_csv('pos_last_try.csv', index_col=0)


In [8]:
dataframe = pos_tags
dataframe = pd.concat([pos_tags,man_feats], axis=1)
# Uncomment to use biling data
#dataframe = pd.concat([dataframe, biling_samples], axis=0, ignore_index = True)
labels = dataframe['labels'].values
data = dataframe.drop('labels', axis=1).values
print data.shape
print labels.shape

(20078, 1066)
(20078,)


In [20]:
t0 = time()
# Feature Extraction
pca = PCA(n_components = 900)
data_decomposed = pca.fit_transform(data)
print("done in %0.3fs." % (time() - t0))

done in 23.854s.


In [10]:
estimators = [('lda', LinearDiscriminantAnalysis()),  ('logReg', LogisticRegressionCV()),
              ('adaB', AdaBoostClassifier(base_estimator=None, learning_rate=1, n_estimators=50))]
classfVC = VotingClassifier(estimators= estimators, voting='hard')

In [21]:
classf = classfVC
train_data, test_data, train_labels, test_labels = train_test_split(data_decomposed,
                                                                    labels, test_size=0.20, random_state=42)

t0 = time()
classf.fit(train_data,train_labels)
print "Score of Classifier: " + str(classf.score(test_data,test_labels))
print("done in %0.3fs." % (time() - t0))

Score of Classifier: 0.546812749004
done in 187.201s.


In [23]:
# Train final Classifier:

classf = classfVC
#classf.fit(data,labels)

In [42]:
test = pd.read_csv("test_blind.txt", sep="\t", header=None, names=['labels','phrases'],
                   lineterminator='\n', quoting=csv.QUOTE_NONE)


In [43]:
test_man_feats = pd.read_csv('test_man.csv', index_col=0)
test_pos_tags = pd.read_csv('test_data.csv', index_col=0)
#test_pos_tags.apply(sum)

In [44]:
dataframe = pd.concat([test_pos_tags,test_man_feats], axis=1)
# Uncomment to use biling data
#dataframe = pd.concat([dataframe, biling_samples], axis=0, ignore_index = True)
test_data = dataframe.drop('labels', axis=1).values
print test_data.shape
test_data_decomposed = pca.transform(test_data)
print test_data_decomposed.shape


(3220, 1066)
(3220, 900)


In [45]:
testlabs = classf.predict(test_data_decomposed)
test.labels = test['labels'] = testlabs
test.to_csv("test_output.txt", sep="\t", header=False, index=False, quoting=csv.QUOTE_NONE, enconding = 'utf-8')

In [49]:
sum(testlabs)*1.0/3220


0.47857142857142859